# Ensemble (averaging)
將三份檔案的預測結果進行平均，再取期望值(prob^1.5 * price)的前三名作為預測輸出。

In [1]:
import numpy as np
import pandas as pd

In [2]:
prob_1 = np.load('data/pred_prob_lgb_2var_customer+_card_v2+offon_card_deze_dt19_24_fe_1k.npy')
prob_2 = np.load('data/pred_prob_lgb_2var_customer+_card_v2+offon_cnt_card_cnt_deze_dt19_24_fe_1k.npy')
prob_3 = np.load('data/pred_prob_lgb_2var_customer+_card_v2+offon_cnt_card_deze_dt19_24_fe_1k.npy')

price_1 = np.load('data/pred_price_lgb_2var_customer+_card_v2+offon_card_deze_dt19_24_fe_1k.npy')
price_2 = np.load('data/pred_price_lgb_2var_customer+_card_v2+offon_cnt_card_cnt_deze_dt19_24_fe_1k.npy')
price_3 = np.load('data/pred_price_lgb_2var_customer+_card_v2+offon_cnt_card_deze_dt19_24_fe_1k.npy')


In [3]:
mean_prob = np.mean([prob_1, prob_2, prob_3], axis = 0)
mean_price = np.mean([price_1, price_2, price_3], axis = 0)

In [4]:
pred_class = ['2', '6', '10', '12', '13', '15', '18', '19', '21', '22', '25', '26', '36', '37', '39', '48']
idx_to_class = {i:class_ for i, class_ in enumerate(pred_class)}

In [5]:
submit = []
prediction = mean_price*(mean_prob**1.5)
for row in prediction:
    submit.append(row.argsort()[-3:][::-1][np.newaxis,:])
submit = np.vstack(submit)

In [6]:
shape = submit.shape
submit = submit.flatten()
submit = np.array([idx_to_class[idx] for idx in submit])
submit = submit.reshape(shape)

In [7]:
submit_csv = pd.read_csv("sample_submission.csv")
submit_csv = submit_csv.sort_values("chid")

In [8]:
submit_csv.iloc[:,1:] = submit

In [9]:
submit_csv

,chid,top1,top2,top3
224333,10000000,10,12,48
57948,10000001,48,12,37
14558,10000002,37,22,2
239971,10000003,37,22,2
384821,10000004,37,36,12
...,...,...,...,...
3396,10499995,2,10,15
283590,10499996,36,15,48
128151,10499997,36,48,25
426218,10499998,10,39,19


In [10]:
submit_csv.to_csv('ensemble_0103.csv', index = False)